In [1]:
import sys
import os
import matplotlib.pyplot as plt

# Add the src directory to the path. TEMPORARY FIX
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../../..")))

from models.duck_net.duck_net import DuckNet
from src.models.data_management.cnn_formes import CNNFormes
from src.data_processing.dataset_loader import CoastData
from src.models.data_management.data_loader import DataLoaderManager

import cv2

/opt/conda/lib/python3.10/site-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.5' (you have '2.0.4'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
# Load the data to split it and save it to a dict
path = os.path.abspath(os.path.join(os.getcwd(), "../../../data/patchify_shoreline/"))
# data = CoastData(data_path=path)

# data_split = data.split_data()

In [3]:
model = DuckNet(num_classes=1, experiment_name="test_experiments", use_mlflow=True)

In [4]:
# Load the data to the model
data = model.load_data(path, CNNFormes)

In [5]:
# Total non-shoreline pixels:  1155193552
# Total shoreline pixels:  4269360
# Total pixels:  1159462912
# Shoreline percentage:  0.3682187636890968

total_pixels = 1159462912
shoreline_pixels = 4269360

# Calculate the class weight
weight = total_pixels / shoreline_pixels

print(weight)

271.577686585343


In [ ]:
artifact_path = os.path.abspath(os.path.join(path, "../../artifacts/"))
print(artifact_path)
# Train the model
model.train(epochs=100, loss_function_name="BCEWithLogitsLoss", weight=1, artifact_path=artifact_path, run_name = "binary_ducknet")

/srv/Shoreline-extraction/artifacts
Creating folders for the artifacts at /srv/Shoreline-extraction/artifacts/2025-03-27-09-13-41_binary_ducknet
Epoch 1/100
train metrics: 
	train_loss: 0.6992455554008484
	train_accuracy: 0.004422612488269806
	train_f1_score: 0.007334501016885042
	train_precision: 0.0036807553842663765
	train_recall: 0.9995608329772949
	train_confusion_matrix: 
		0.0007 0.9993
		0.0004 0.9996

validation metrics: 
	validation_loss: 0.6931753471426295
	validation_accuracy: 0.004024001304060221
	validation_f1_score: 0.007352471351623535
	validation_precision: 0.003689802484586835
	validation_recall: 0.9999390244483948
	validation_confusion_matrix: 
		0.0003 0.9997
		0.0001 0.9999

Validation loss improved from inf to 0.693175. Saving the model. Early stopping counter: 0/25
Epoch 2/100
train metrics: 
	train_loss: 0.6931621454608056
	train_accuracy: 0.004260841757059097
	train_f1_score: 0.007334339432418346
	train_precision: 0.0036806724965572357
	train_recall: 0.99970108

# Predictions

In [ ]:
import numpy as np
data = DataLoaderManager.load_data(os.path.abspath(os.path.join(os.getcwd(), "../../data/patchify_shoreline/")))
index = 1
img_path = data["test"]['images'][index]
mask_path = data["test"]['masks'][index]

artifact_path = os.path.abspath(os.path.join(os.getcwd(), "../../../artifacts/"))
model.load_model(os.path.abspath(os.path.join(artifact_path, "2025-03-25-09-00-40_binary_ducknet/models/best_model.pth")))

img = cv2.imread(img_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

pred = model.predict(img_path)

# print(np.unique(mask, return_counts=True))
# print(np.unique(pred, return_counts=True))

fig, ax = plt.subplots(1, 3, figsize=(15, 5))
ax[0].imshow(img)
ax[1].imshow(mask)
ax[2].imshow(pred.cpu().squeeze())

plt.show()